In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from fredapi import Fred
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import fredpy as fp

In [2]:
# .env에서 api_key 가져오기 
load_dotenv()

FRED_API_KEY = os.getenv('FRED_API_KEY')

# fredapi 
fred = Fred(api_key = FRED_API_KEY)


In [3]:
# 실업률
unemployment = fred.get_series_all_releases('UNRATE')

# PPI
ppi = fred.get_series_all_releases('PPIACO')
ppi.tail()

,realtime_start,date,value
1710,2024-08-13 00:00:00,2024-06-01 00:00:00,255.779
1711,2024-09-12 00:00:00,2024-06-01 00:00:00,256.015
1712,2024-08-13 00:00:00,2024-07-01 00:00:00,257.723
1713,2024-09-12 00:00:00,2024-07-01 00:00:00,257.485
1714,2024-09-12 00:00:00,2024-08-01 00:00:00,255.613


In [4]:
# fredpy
plt.style.use('classic')
plt.rcParams.update({'figure.facecolor': 'white'})

fp.api_key = FRED_API_KEY


## 예시 
CPI 데이터 가져와서 시각화

In [5]:
# sample : CPI
enddate=(datetime.now() + timedelta(days=-1)).strftime('%Y-%m-%d')
cpi= fp.series('CPIAUCSL',enddate)
corecpi = fp.series('CPILFESL',enddate)
cpi.data
cpiyoy=cpi.apc().data
cpiyoy
cpimom=cpi.data.pct_change()*100
cpimom
corecpiyoy=corecpi.apc().data
corecpimom=corecpi.data.pct_change()*100

import plotly.express as px

fig=px.bar(x=cpiyoy.index, y=cpiyoy.values, color=cpiyoy.values, 
          title='CPI YOY 장기데이터', height=500)
fig.update_traces(textposition='outside')
fig.update_layout(
    uniformtext_minsize=8, uniformtext_mode='hide',
    title={
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [6]:
dfyoy = pd.DataFrame({'CPI':cpiyoy.values, 'coreCPI':corecpiyoy},index=cpiyoy.index)
dfyoy

,CPI,coreCPI
date,,
1948-01-01,10.242086,NaN
1948-02-01,9.481961,NaN
1948-03-01,6.818182,NaN
1948-04-01,8.272727,NaN
1948-05-01,9.384966,NaN
...,...,...
2024-04-01,3.357731,3.615527
2024-05-01,3.250210,3.411063
2024-06-01,2.975629,3.276939


In [7]:
import plotly.express as px

fig=px.line(data_frame=dfyoy, x=dfyoy.index, y=['CPI','coreCPI'], 
          title='CPI vs CoreCPI YOY 장기데이터', height=500)
fig.update_layout(
    uniformtext_minsize=8, uniformtext_mode='hide',
    title={
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [8]:
# 경기 침체

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

usreg = fp.series('USREC',enddate)
usregdata=usreg.data.loc[usreg.data.index>='1945-01-01']

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Line(x=dfyoy.index, y=dfyoy['CPI'], name="CPI"),
    secondary_y=True,
)

fig.add_trace(
    go.Line(x=dfyoy.index, y=dfyoy['coreCPI'], name="Core CPI"),
    secondary_y=True,
)

fig.add_trace(
    go.Bar(x=usregdata.index, y=usregdata.values, name="경기침체"),
    secondary_y=False,
)

fig.show()

c:\Users\Skku\anaconda3\envs\economy\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## 시작
### 데이터 수집 목록
- 물가 : CPI, PCE, PPI
- 고용 : 실업률, 비농업 고용지수, 평균 시간당 임금, 실업수당 청구 건수, JOLTs Job Openings, 노동수요대비 공급률
- 경기 : GDP 성장률, PMI, Retail Sales, 산업 생산 지수, OECD 경기선행지수, 소비자 심리 지수, Citi Economic Surprise Index
* Fred에 없는 지표 : 노동수요대비공급률, PMI, citi economic surprise index

In [20]:
# PPIfd : PPI final demand
# AHE : Average Hourly Earnings
# GDP : real GDP growth rate 

code_dic = {'CPI': 'CPIAUCSL', 'coreCPI':'CPILFESL', 'PCE':'PCE', 'corePCE':'PCEPI', 'PPI':'PPIACO', 'PPIfd':'PPIFIS', 'corePPIfd':'PPIFES', 
            'unemployment':'UNRATE', 'Nonfarm':'PAYEMS', 'AHE':'CES0500000003', 'initial claims': 'ICSA','continued claims':'CCSA', 'job openings':'JTSJOL',
            'GDP':'A191RL1Q225SBEA', 'retail':'MARTSMPCSM44X72USS', 'industrial production':'INDPRO', 'OECD leading':'USALOLITONOSTSAM','consumer sentiment':'UMCSENT'}

IR_code = 'FEDFUNDS'

code_dic.values()

dict_values(['CPIAUCSL', 'CPILFESL', 'PCE', 'PCEPI', 'PPIACO', 'PPIFIS', 'PPIFES', 'UNRATE', 'PAYEMS', 'CES0500000003', 'ICSA', 'CCSA', 'JTSJOL', 'A191RL1Q225SBEA', 'MARTSMPCSM44X72USS', 'INDPRO', 'USALOLITONOSTSAM', 'UMCSENT'])

In [22]:
# fredpy로 max 기간만큼 가져오기

series_dic = {}

for key in code_dic.keys() : 
     series_dic[key] = fp.series(code_dic[key],enddate)

In [23]:
# 데이터프레임 만들기 위해 각 데이터셋의 date 확인
# initial claims, continues claims 만 각 월 여러번 같은 날
# 나머지 각 월 1일에 

for key in series_dic.keys() :      
    print(f'{key} : {series_dic[key].data.index}')

CPI : DatetimeIndex(['1947-01-01', '1947-02-01', '1947-03-01', '1947-04-01',
               '1947-05-01', '1947-06-01', '1947-07-01', '1947-08-01',
               '1947-09-01', '1947-10-01',
               ...
               '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01',
               '2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01',
               '2024-07-01', '2024-08-01'],
              dtype='datetime64[ns]', name='date', length=932, freq='MS')
coreCPI : DatetimeIndex(['1957-01-01', '1957-02-01', '1957-03-01', '1957-04-01',
               '1957-05-01', '1957-06-01', '1957-07-01', '1957-08-01',
               '1957-09-01', '1957-10-01',
               ...
               '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01',
               '2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01',
               '2024-07-01', '2024-08-01'],
              dtype='datetime64[ns]', name='date', length=812, freq='MS')
PCE : DatetimeIndex(['1959-01-01', '1959-02-01', '1959-0

In [24]:
# initial claims, continued claims 만 한 달에 여러번 같은 날에 발표
# 나머지 매 월 1일에 발표
# claims : 같은 년월에 발표된 값들을 평균화해서 하나로 만들겠다.

initial = series_dic['initial claims'].data
initial = pd.DataFrame({'date':initial.index,'value':initial.values})
continued = series_dic['continued claims'].data
continued = pd.DataFrame({'date':continued.index,'value':continued.values})
claims = initial.merge(continued, how='outer', on='date', suffixes=('initial claims','continued claims'))
claims

,date,valueinitial claims,valuecontinued claims
0,1967-01-07,208000.0,1134000.0
1,1967-01-14,207000.0,1119000.0
2,1967-01-21,217000.0,1119000.0
3,1967-01-28,204000.0,1103000.0
4,1967-02-04,216000.0,1131000.0
...,...,...,...
3006,2024-08-17,233000.0,1860000.0
3007,2024-08-24,232000.0,1845000.0
3008,2024-08-31,228000.0,1843000.0
3009,2024-09-07,231000.0,1829000.0


In [25]:
dformat = '%Y-%m'
temp = claims['date'][0]


#datetime.strftime('%Y-%m')
claims_new = claims.copy()

claims_new['date'] = pd.to_datetime(claims_new['date'])
claims_new['date'] = claims_new['date'].dt.strftime('%Y-%m-01')
claims_new


,date,valueinitial claims,valuecontinued claims
0,1967-01-01,208000.0,1134000.0
1,1967-01-01,207000.0,1119000.0
2,1967-01-01,217000.0,1119000.0
3,1967-01-01,204000.0,1103000.0
4,1967-02-01,216000.0,1131000.0
...,...,...,...
3006,2024-08-01,233000.0,1860000.0
3007,2024-08-01,232000.0,1845000.0
3008,2024-08-01,228000.0,1843000.0
3009,2024-09-01,231000.0,1829000.0


In [26]:
claims_new = claims_new.groupby(['date']).mean()
claims_new

,valueinitial claims,valuecontinued claims
date,,
1967-01-01,209000.0,1118750.0
1967-02-01,229000.0,1162500.0
1967-03-01,260750.0,1243250.0
1967-04-01,263000.0,1281000.0
1967-05-01,235750.0,1277500.0
...,...,...
2024-05-01,223000.0,1788500.0
2024-06-01,236800.0,1833800.0
2024-07-01,238250.0,1861000.0


In [39]:
# 1. 모든 지표 병합
# 2. 모든 지표 YoY, MoM 연산 후 컬럼으로 추가
# 3. 금리는 전 발표 대비 변동

ir = fp.series('FEDFUNDS',enddate)

series_dic['CPI'].apc().data, series_dic['CPI'].data.pct_change()*100

# for key in series_dic.keys():
#     yoy = series_dic[key].data.apc()

(date
 1948-01-01    10.242086
 1948-02-01     9.481961
 1948-03-01     6.818182
 1948-04-01     8.272727
 1948-05-01     9.384966
                 ...    
 2024-04-01     3.357731
 2024-05-01     3.250210
 2024-06-01     2.975629
 2024-07-01     2.923566
 2024-08-01     2.591227
 Freq: MS, Name: value, Length: 920, dtype: float64,
 date
 1947-01-01         NaN
 1947-02-01    0.651769
 1947-03-01    1.757632
 1947-04-01    0.000000
 1947-05-01   -0.227273
                 ...   
 2024-04-01    0.312910
 2024-05-01    0.005747
 2024-06-01   -0.056190
 2024-07-01    0.154928
 2024-08-01    0.187221
 Freq: MS, Name: value, Length: 932, dtype: float64)